Testing how using different priors on the slope parameters changes posteriors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import corner
import emcee

Define various functions

In [ ]:
def model(m, c, x):
    return m * x + c

Priors

In [ ]:
def simple_lnprior(pars):
    if -10 < pars[0] < 10 and -10 < pars[1] < 10 and -10 < pars[2] < 10:
        return 0.
    else:
        return -np.inf

In [ ]:
def jake_lnprior(pars):
    alpha, beta, sigma = pars
    if sigma < 0:
        return -np.inf
    else:
        return -1.5 * np.log(1 + beta ** 2) - np.log(sigma)

Log probs

In [ ]:
def simple_lnprob(pars, x, y, yerr):
    return simple_lnprior(pars) + lnlike(pars, x, y, yerr)

In [ ]:
def jake_lnprob(pars, x, y, yerr):
    return jake_lnprior(pars) + lnlike(pars, x, y, yerr)

Likelihood

In [ ]:
def lnlike(pars, x, y, yerr):
    m, c, s = pars
    invsig2 = 1./(yerr**2 + np.exp(2*s))
    model_y = model(m, c, x)
    return -.5*np.sum((y-model_y)**2*invsig2 - np.log(invsig2))

In [ ]:
# load data
f, ferr, r, rerr = np.genfromtxt("../data/flickers.dat").T

# fit a line
AT = np.vstack((r, np.ones_like(r)))
ATA = np.dot(AT, AT.T)
m, c = np.linalg.solve(ATA, np.dot(AT, f))
print("params = ", m, c)

# plot data with best fit line
xs = np.linspace(min(r), max(r), 100)
ys = m * xs + c
plt.errorbar(r, f, xerr=rerr, yerr=ferr, fmt="k.", capsize=0)
plt.plot(xs, ys)
plt.ylabel("log flicker")
plt.xlabel("log rho")

Run simple version

In [ ]:
pars_init = [m, c, np.log(3)]
print(simple_lnlike(pars_init, r, f, ferr)) # check the lhf works

In [ ]:
ndim, nwalkers = 3, 24
pos = [pars_init + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]  # initialisation
sampler = emcee.EnsembleSampler(nwalkers, ndim, uniform_lnprob, args=(r, f, ferr))
sampler.run_mcmc(pos, 10000)  # run MCMC
samples = sampler.chain[:, 1000:, :].reshape((-1, ndim))  # cut off burn in and flatten chains
samples[:, 2] = np.exp(samples[:, 2])
m1, c1, ln_sig1 = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
fig = corner.corner(samples, labels=["m", "c", "s"])

Run uniform in cos theta version

In [ ]:
pars_init = [np.cos(np.arctan(m)), c, np.log(3)]
print(angle_lnlike(pars_init, r, f, ferr)) # check the lhf works
print(angle_lnprob(pars_init, r, f, ferr)) # check the lhf works
print(angle_lnprior(pars_init)) # check the lhf works

In [ ]:
ndim, nwalkers = 3, 24
pos = [pars_init + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]  # initialisation
sampler = emcee.EnsembleSampler(nwalkers, ndim, angle_lnprob, args=(r, f, ferr))
sampler.run_mcmc(pos, 10000)  # run MCMC
samples = sampler.chain[:, 1000:, :].reshape((-1, ndim))  # cut off burn in and flatten chains
samples[:, 2] = np.exp(samples[:, 2])
samples[:, 0] = np.tan(np.arccos(samples[:, 0]))
m2, c2, ln_sig2 = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
fig = corner.corner(samples, labels=["m", "c", "s"])